In [ ]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr
import litellm

In [ ]:
# This one line reads your .env file and loads your GEMINI_API_KEY
load_dotenv()

print("✅ API Key loaded securely from .env file.")

In [ ]:
# Initialize
MODEL = 'gemini/gemini-1.5-flash'

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = litellm.completion(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# Gradio magic 
# gr.ChatInterface(fn=chat, type="messages").launch() #chatinterface only work when function is written in chat(message,history) format

In [ ]:
# now Multi-shot promt
system_message = "You are a helpful assistant in a bookstore. You should gently encourage customers to explore titles \
    that are currently on sale. \
    Fiction books are 60% off, and most other genres are 50% off. \
    For example, if a customer says 'I'm looking to buy a novel', you could reply something like,\
    'Excellent choice — we have a wide selection of novels, including many featured in our current discount event.\
    ' If the customer seems unsure, kindly suggest popular or discounted fiction titles to help them decide."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = litellm.completion(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for children's books, you should respond that children's books are not on sale today, \
but kindly remind them to check out our fiction section, where many titles are currently 60% off."


In [ ]:
#gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# reply for a perticular keyword.
def chat(message, history):

    relevant_system_message = system_message
    if 'newspaper' in message:
        relevant_system_message += " The store does not sell newspaper ; if you are asked for newspaper, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = litellm.completion(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()